In [ ]:
# Import packages 
import cv2
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join

#sort with 2nd value
def sort_second(val): 
    return val[1]  

def Average(lst): 
    return sum(lst) / len(lst) 

def Bg():
    black_img = np.zeros(shape=[75, 75], dtype=np.uint8)
    bg_img = cv2.bitwise_not(black_img)
    return bg_img

def Merge_label(stats, img, wpath, file, num):
    min_x = min_y = 200
    max_x = max_y = 0
    for c in stats:
        if min_x > c[1]:
            min_x = c[1]
        if min_y > c[0]:
            min_y = c[0]
        if max_x < c[1]+c[3]:
            max_x = c[1]+c[3]
        if max_y < c[0]+c[2]:
            max_y = c[0]+c[2]
    print (min_y,min_x, max_y, max_x)
    bg_img = Generate_image(min_y,min_x, max_y, max_x, img)
    path = wpath + "/" + str(num) + "/" + file
    print(path)
    cv2.imwrite(path,bg_img)
    return

def Generate_image(min_y, min_x, max_y, max_x, img):
#    bg_img = Bg()
    if (max_x - min_x) > 75:
        bg_img = Hresize(min_y, min_x, max_y, max_x, img)
    elif (max_y - min_y) > 75:
        bg_img = Vresize(min_y, min_x, max_y, max_x, img)
    else:
        top_left_x = int(round((75 - max_x + min_x)/2))
        top_left_y = int(round((75 - max_y + min_y)/2))
        dx = top_left_x + (max_x - min_x)
        dy = top_left_y + max_y - min_y
        # print(top_left_y, dy, top_left_x, dx, min_y, max_y, min_x, max_x)

#        bg_img[top_left_y:dy, top_left_x:dx] = img[min_y:max_y, min_x:max_x]
        bg_img = img[min_y:max_y, min_x:max_x]
#    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(bg_img)
#    fig, ax = plt.subplots(facecolor="w")
#    ax.imshow(labels)
#    plt.show()
#    num, confidence = Font_recognition(bg_img)
    return bg_img

def Hresize(min_y, min_x, max_y, max_x, img):
    dsize = (75, int(round((max_y-min_y)*75/(max_x-min_x))))
    resized = cv2.resize(img[min_y:max_y, min_x:max_x], dsize)
    h,w = resized.shape
    top_left_x = int(round((75 - w)/2))
    top_left_y = int(round((75 - h)/2)-1)
    dx = top_left_x + w
    dy = top_left_y + h
    print(top_left_y, dy, top_left_x, dx, h, w)
    bg_img = Bg()
    bg_img[top_left_y:dy, top_left_x:dx] = resized
    return bg_img

def Vresize(min_y, min_x, max_y, max_x, img):
    dsize = (int(round((max_x-min_x)*75/(max_y-min_y))),75)
    resized = cv2.resize(img[min_y:max_y, min_x:max_x], dsize)
    h,w = resized.shape
    top_left_x = int(round((75 - w)/2))
    top_left_y = int(round((75 - h)/2))
    dx = top_left_x + w
    dy = top_left_y + h
    print(top_left_y, dy, top_left_x, dx, h, w)
    bg_img = Bg()
    bg_img[0:dy, top_left_x:dx] = resized
    return bg_img

def Find_number_of_digits(fname):
    number_digit = 0
    number = ""
    for c in fname:
        if c == "_":
            break
        else:
            number_digit += 1
            number = number + c
    return number_digit, number
        

def Extract_labels(npath, file, wpath, nd, n):
    number_digit = nd
    group = []
    # Read image in grayscale
    img = cv2.imread(npath +"/"+ file, 0)
    #cv2.imshow("otsu", img)
    #cv2.waitKey()
    #cv2.destroyAllWindows()

    # Convert the grayscale to black and white. CV2 sets teh threshhold automatically.
    ret2, img_otsu = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    # print("ret2: {}".format(ret2))

    #　Reverse black and white
    src = cv2.bitwise_not(img_otsu)

    # labeling
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(src)
    fig, ax = plt.subplots(facecolor="w")
    ax.imshow(labels)
    plt.show()

    # Number of labels
    #print("number of labels:", retval)  # 6 (background + objects)
    # labeling result
    #print(labels.shape, labels.dtype)  # (362, 420) int32
    #print(stats.shape, stats.dtype)

    # delete background (label number 0) and objects with less than or equal to 50 pix from the extracted labels
    new_stats = []
    for i, row in enumerate(stats):
    #    print(f"label {i}")
    #    print(f"  topleft: ({row[cv2.CC_STAT_LEFT]}, {row[cv2.CC_STAT_TOP]})")
    #    print(f"  size: ({row[cv2.CC_STAT_WIDTH]} px, {row[cv2.CC_STAT_HEIGHT]} px)")
    #    print(f"  area: {row[cv2.CC_STAT_AREA]}")
        if i == 0:
            continue
        if row[cv2.CC_STAT_AREA] > 50:
            new_stats.append([row[cv2.CC_STAT_TOP],row[cv2.CC_STAT_LEFT],
                              row[cv2.CC_STAT_HEIGHT],row[cv2.CC_STAT_WIDTH],row[cv2.CC_STAT_AREA]])
        # Display the extracted results
        # Background image to put the labels on 
        #    bg_img = np.zeros(shape=[75, 75], dtype=np.uint8)
        #    top_left_x = int(round((75 - row[cv2.CC_STAT_WIDTH])/2))
        #    top_left_y = int(round((75 - row[cv2.CC_STAT_HEIGHT])/2))
        #    dx = top_left_x + int(row[cv2.CC_STAT_WIDTH])
        #    dy = top_left_y + int(row[cv2.CC_STAT_HEIGHT])
        #    x = int(row[cv2.CC_STAT_LEFT]) + int(row[cv2.CC_STAT_WIDTH])
        #    y = int(row[cv2.CC_STAT_TOP]) + int(row[cv2.CC_STAT_HEIGHT])

        #    bg_img[top_left_y:dy, top_left_x:dx] = src[row[cv2.CC_STAT_TOP]:y, row[cv2.CC_STAT_LEFT]:x]
        #    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(bg_img)
        #    fig, ax = plt.subplots(facecolor="w")
        #    ax.imshow(labels)
        #    plt.show()
#print(new_stats)
#　Group labels if number of the labels are greater than 2 when the number is 2-digits.
    if len(new_stats) > 1 and number_digit == 2:
    # Calculate distances between lables
        dl = []
        k = 0
        for i in new_stats:
            d = 0
            for j in new_stats:
                if i == j:
                    continue
                else:
                    d = (i[1]+i[3]/2-j[1]-j[3]/2)**2 + d
            de = (k, d)
            dl.append(de)
            k += 1
        print(dl)
        # sort the distance from larer to smaller
        dl.sort(key = sort_second, reverse = True)
        print(dl)

        # group labels
        group = [[dl[0][0]],[dl[len(dl)-1][0]]]
    #    print(group)
        # set distance of labels of two-digits as the default
        d = dl[0][1] - dl[1][1]
        gn = 0
        #　making labels in one group based on the distance 
        for i in range(1,len(dl)-1,):
            ds = dl[i][1] - dl[i+1][1]
            if ds < d:
                gn = 1
                group[gn].append(dl[i][0])
            else:
                group[gn].append(dl[i][0])
            d = ds
#    print (group)

    # Merge lebels
    if number_digit == 1 and len(new_stats) >= 1:
        Merge_label(new_stats, img, wpath, file, n)
    elif number_digit > 1 and len(new_stats) > 1:
        print("nd:{},new_stats{},group{},retval{},file{}".format(nd,new_stats,group, retval, file))
        av = []
        for i in range(0,2):
            xp = []
            if len(group) < 2:
                continue
            else:
                for d in group[i]:
                    xp.append(new_stats[d][1])
                av.append(Average(xp))
                print(av)
        if av[0]<av[1]:
            stats = []
            for d in group[0]:
                stats.append(new_stats[d])
            num = int(n)//10
            Merge_label(stats, img, wpath, file, num)
            stats = []
            for d in group[1]:
                stats.append(new_stats[d])
            num = int(n)%10
            Merge_label(stats, img, wpath, file, num)
        else:
            stats = []
            for d in group[1]:
                stats.append(new_stats[d])
            num = int(n)//10
            Merge_label(stats, img, wpath, file, num)
            stats = []
            for d in group[0]:
                stats.append(new_stats[d])
            num = int(n)%10
            Merge_label(stats, img, wpath, file, num)
    else:
        pass

#    Main logic            
rpath = "F:/Google Drive/User_Backup/scanned"
wpath = "F:/Google Drive/User_Backup/base_font"
#onlyfiles = [f for f in listdir(rpath) if isfile(join(mypath, f)) and f[-4:] == ".png"]
onlyfolders = [f for f in listdir(rpath)]
for folder in onlyfolders:
    if folder[0] == "A" or folder[0] == "M" or folder[0] == "S":
        print(rpath+"/"+folder)
        npath = rpath +"/"+ folder
        onlyfiles = [f for f in listdir(npath) if isfile(join(npath, f)) and f[-4:] == ".png"]
        for file in onlyfiles:
            nd, n = Find_number_of_digits(file)
            Extract_labels(npath, file, wpath, nd, n)
